In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housing-prices-competition-for-kaggle-learn-users/train.csv
/kaggle/input/housing-prices-competition-for-kaggle-learn-users/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_train = pd.read_csv('../input/housing-prices-competition-for-kaggle-learn-users/train.csv', index_col='Id') 
X_test = pd.read_csv('../input/housing-prices-competition-for-kaggle-learn-users/test.csv', index_col='Id')

# Remove rows with missing target 
X_train.dropna(axis=0, subset=['SalePrice'], inplace=True)

#separate target from predictors
y_train = X_train.SalePrice
X_train.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_2, X_valid, y_train_2, y_valid = train_test_split(X_train, y_train, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [3]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [4]:
#Question 1
from sklearn.impute import SimpleImputer

drop_X_train_2 = X_train_2.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

imputer = SimpleImputer(strategy='median')
imputed_X_train_2 = pd.DataFrame(imputer.fit_transform(drop_X_train_2))
imputed_X_valid = pd.DataFrame(imputer.transform(drop_X_valid))

imputed_X_train_2.columns = drop_X_train_2.columns
imputed_X_valid.columns = drop_X_valid.columns

print("Mean Absolute Error from dropping categorical columns and imputing missing values:")
print(score_dataset(imputed_X_train_2, imputed_X_valid, y_train_2, y_valid))

Mean Absolute Error from dropping categorical columns and imputing missing values:
17608.601228060787


Mean Absolute Error from dropping categorical columns and imputing missing values:
17608.601228060787

In [5]:
#Question 2
from xgboost import XGBRegressor

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_test = X_test.select_dtypes(exclude=['object'])

imputer = SimpleImputer(strategy='median')
imputed_X_train = pd.DataFrame(imputer.fit_transform(drop_X_train))
imputed_X_test = pd.DataFrame(imputer.transform(drop_X_test))

imputed_X_train.columns = drop_X_train.columns
imputed_X_test.columns = drop_X_test.columns

model = XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=0)
model.fit(imputed_X_train, y_train)

preds_test = model.predict(imputed_X_test)
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

Score: 15976.43314

In [6]:
#Question 3
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

object_cols=[col for col in X_train_2.columns if X_train_2[col].dtype == "object"]
good_label_cols = [col for col in object_cols if set(X_valid[col]).issubset(set(X_train_2[col]))]
        
bad_label_cols = list(set(object_cols)-set(good_label_cols))

from sklearn.preprocessing import OrdinalEncoder

ordinal_encoded_X_train_2 = X_train_2.drop(bad_label_cols, axis=1)
ordinal_encoded_X_valid= X_valid.drop(bad_label_cols, axis=1)

ordinal_encoder = OrdinalEncoder()
ordinal_encoded_X_train_2[good_label_cols] = ordinal_encoder.fit_transform(X_train_2[good_label_cols])
ordinal_encoded_X_valid[good_label_cols] = ordinal_encoder.transform(X_valid[good_label_cols])

my_imputer = SimpleImputer(strategy="median")
imputed_X_train_2= pd.DataFrame(my_imputer.fit_transform(ordinal_encoded_X_train_2))
imputed_X_valid = pd.DataFrame(my_imputer.transform(ordinal_encoded_X_valid))

imputed_X_train_2.columns = ordinal_encoded_X_train_2.columns
imputed_X_valid.columns = ordinal_encoded_X_valid.columns

print("Mean absolute Error on validation set")
print(score_dataset(imputed_X_train_2,imputed_X_valid, y_train_2, y_valid))

Mean absolute Error on validation set
16501.22931827911


Mean absolute Error on validation set
16501.22931827911

In [7]:
# Question 4
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

object_cols=[col for col in X_train.columns if X_train[col].dtype == "object"]
good_label_cols = [col for col in object_cols if set(X_test[col]).issubset(set(X_train[col]))]
         
bad_label_cols = list(set(object_cols)-set(good_label_cols))

ordinal_encoded_X_train = X_train.drop(bad_label_cols, axis=1)
ordinal_encoded_X_test= X_test.drop(bad_label_cols, axis=1)

ordinal_encoder = OrdinalEncoder()
ordinal_encoded_X_train[good_label_cols] = ordinal_encoder.fit_transform(X_train[good_label_cols])
ordinal_encoded_X_test[good_label_cols] = ordinal_encoder.transform(X_test[good_label_cols])

my_imputer = SimpleImputer(strategy="median")
imputed_X_train= pd.DataFrame(my_imputer.fit_transform(ordinal_encoded_X_train))
imputed_X_test = pd.DataFrame(my_imputer.transform(ordinal_encoded_X_test))

imputed_X_train.columns = ordinal_encoded_X_train.columns
imputed_X_test.columns = ordinal_encoded_X_test.columns

model.fit(imputed_X_train, y_train)

test_preds = model.predict(imputed_X_test)

output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': test_preds})
output.to_csv('submission2.csv', index=False)

Score: 14664.82491

In [8]:
# Question 5
from sklearn.preprocessing import OneHotEncoder

low_cardinality_cols = [col for col in X_train_2.columns if X_train_2[col].nunique() < 10 and X_train_2[col].dtype == "object"]

onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_X_train_2 = pd.DataFrame(onehot_encoder.fit_transform(X_train_2[low_cardinality_cols]))
OH_X_valid = pd.DataFrame(onehot_encoder.transform(X_valid[low_cardinality_cols]))

X_train_2_index = X_train_2.drop(low_cardinality_cols, axis=1)
X_valid_index = X_valid.drop(low_cardinality_cols, axis=1)
OH_X_train_2.index = X_train_2_index.index
OH_X_valid.index = X_valid_index.index

numeric_X_train_2 = X_train_2.select_dtypes(exclude=['object'])
numeric_X_valid = X_valid.select_dtypes(exclude=['object'])

OH_X_train_2 = pd.concat([numeric_X_train_2, OH_X_train_2], axis=1)
OH_X_valid = pd.concat([numeric_X_valid, OH_X_valid], axis=1)

OH_X_train_2.columns = OH_X_train_2.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

imputer = SimpleImputer(strategy='median')
imputed_X_train_2 = pd.DataFrame(imputer.fit_transform(OH_X_train_2))
imputed_X_valid = pd.DataFrame(imputer.transform(OH_X_valid))

imputed_X_train_2.columns = OH_X_train_2.columns
imputed_X_valid.columns = OH_X_valid.columns

print("Mean Absolute Error on the validation set:")
print(score_dataset(imputed_X_train_2, imputed_X_valid, y_train_2, y_valid))

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Mean Absolute Error on the validation set:
16625.319670376713


Mean Absolute Error on the validation set:
16625.319670376713

In [16]:
# Question 6
from sklearn.preprocessing import OneHotEncoder
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_X_train =pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_X_test =pd.DataFrame(OH_encoder.transform(X_test[low_cardinality_cols])) 

OH_X_train.index = X_train.index
OH_X_test.index = X_test.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_X_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_X_test], axis=1)

OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_test.columns = OH_X_test.columns.astype(str)

my_imputer = SimpleImputer(strategy="median")
imputed_X_train= pd.DataFrame(my_imputer.fit_transform(OH_X_train))
imputed_X_test= pd.DataFrame(my_imputer.transform(OH_X_test))

imputed_X_train.columns = OH_X_train.columns
imputed_X_test.columns = OH_X_test.columns
 
model=XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=0)
model.fit(imputed_X_train, y_train)
preds = model.predict(imputed_X_test)

output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds})
output.to_csv('submission53.csv', index=False)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Score: 15139.54517